In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
%config Completer.use_jedi = False
%matplotlib inline

In [2]:
# ! ls data/redir

In [3]:
# products = pd.read_csv('./data/redir/products.csv', index_col=0)
categories = pd.read_csv('./data/redir/categories.csv', index_col=0)
# products_categories = pd.read_csv("./data/redir/products_categories.csv", index_col=0)
redirs = pd.read_csv("./data/redir/420_redirects.csv", index_col=0)
# with open("./data/redir/sessions_420.json", "r") as f:
#     sessions = json.loads(f.read())

In [4]:
categories.shape, redirs.shape

((2483, 5), (1412, 7))

In [5]:
def category_with_id(idx):
    return categories[categories['id'] == idx]

In [6]:
categories.head(2)

,id,parent_id,site_id,external_id,name
0,3327511,3328184.0,420,1471,Cow
1,3327512,3328184.0,420,1470,Trolls


In [7]:
# ! pip install anytree tqdm
from anytree import Node, RenderTree

In [8]:
cat_dict = dict(categories[['id', 'name']].values.tolist())
cat_dict[-1] = "MAIN_OF_THE_MAIN"

In [9]:
len(categories.drop_duplicates()), len(categories[['id', 'parent_id']].drop_duplicates())

(2483, 2483)

In [10]:
tt = categories[['id', 'parent_id']].drop_duplicates()
tt.fillna(-1, inplace=True)
tt['parent_id'] = tt['parent_id'].apply(int)
tt['name'] = tt['id'].map(cat_dict)
tt['parent_name'] = tt['parent_id'].map(cat_dict)
tt

,id,parent_id,name,parent_name
0,3327511,3328184,Cow,бренды
1,3327512,3328184,Trolls,бренды
2,3327516,3328184,Фельдшер,бренды
3,3658304,3327503,Наборы одеколонов,Ароматы и коллекции
4,3658305,3328459,Искусство дарить подарки,Jo Malone London
...,...,...,...,...
2478,873482,873205,для глаз,кисти и спонжи
2479,873483,873205,для губ,кисти и спонжи
2480,873484,873205,средства для очистки кистей,кисти и спонжи
2481,873485,873205,наборы кистей,кисти и спонжи


In [11]:
tree_dict = {}
for x in tt.values:
    if x[0] not in tree_dict:
        tree_dict[x[0]] = Node((x[0], x[2]))
    if x[1] not in tree_dict:
        tree_dict[x[1]] = Node((x[1], x[3]))

for x in tt.values:
    tree_dict[x[0]].parent = tree_dict[x[1]]

In [12]:
for pre, fill, node in RenderTree(tree_dict[3328697]):
    print("%s%s" % (pre, node.name))
for pre, fill, node in RenderTree(tree_dict[3327983]):
    print("%s%s" % (pre, node.name))

(3328697, 'Giorgio Armani')
├── (3329168, 'Классические ароматы ')
│   ├── (3327639, 'Ароматы для женщин')
│   └── (3327641, 'Ароматы для мужчин')
├── (3329171, 'Уход')
├── (3329173, ' Armani Privé')
│   └── (3328698, 'Коллекция 1001 NUITS')
├── (3328747, 'Новинки')
├── (3328656, 'Бестселлеры ')
└── (3329165, 'Макияж')
    ├── (3327630, 'Губы')
    ├── (3327644, 'Аксессуары')
    ├── (3328709, 'Лицо')
    └── (3327629, 'Глаза')
(3327983, 'NYX Professional Makeup')
├── (3420490, 'Подарочные наборы')
├── (3328885, 'Sale')
├── (3327500, 'Палетки')
├── (3327501, 'Лицо')
├── (3327502, 'Глаза')
├── (3327504, 'Губы')
├── (3329205, 'Новинки')
├── (3329206, 'Аксессуары')
└── (3329374, 'Бестселлеры')


In [13]:
# see it if needed
# for pre, fill, node in RenderTree(tree_dict[-1]):
#     print("%s%s" % (pre, node.name))

In [14]:
redir_cats = pd.merge(redirs[['category_id']], categories[['id', 'parent_id', 'name']], left_on='category_id', right_on='id')#.drop_duplicates()
redir_cats['parent_id'] = redir_cats['parent_id'].apply(int)
redir_cats.head(2)

,category_id,id,parent_id,name
0,3327621,3327621,3327595,Хайлайтеры
1,3327621,3327621,3327595,Хайлайтеры


In [15]:
red_cats = set(redir_cats['category_id'].unique())
red_nodes = {tree_dict[i] for i in red_cats}
{x.depth for x in red_nodes}

{3, 4, 5, 6}

In [16]:
from tqdm import tqdm

new_main = {x for x in red_nodes if x.depth == 3}
small = {x for x in red_nodes if x.depth > 3}
to_3_depth = {}
t3d = set()

for x in tqdm(small):
    temp = x
    while temp.depth > 3:
        temp = temp.parent
    t3d.add(temp)
    to_3_depth[x.name[0]] = temp.name[0]
    #.add(temp)
for x in tqdm(new_main):
    to_3_depth[x.name[0]] = x.name[0]

100%|██████████| 776/776 [00:00<00:00, 965523.56it/s]


In [17]:
list(sorted(Counter(to_3_depth.values()).items(), key=lambda x: -x[1]))[:5]

[(3327548, 41), (3328328, 5), (3327913, 2), (3329394, 2), (3327532, 1)]

In [18]:
redirs['new_category_id'] = redirs['category_id'].map(to_3_depth)
redirs.head()

,redir_id,rule_id,start_date,query,category_id,parent_id,category,new_category_id
0,10303,23358,2020-08-19,dior хайлайтер,3327621,3327595.0,Хайлайтеры,3327548
1,10304,23359,2020-08-19,dior хайлатер,3327621,3327595.0,Хайлайтеры,3327548
2,10319,23374,2020-08-19,diorskin mineral nude luminizer,3327621,3327595.0,Хайлайтеры,3327548
3,10476,23531,2020-08-19,хайдайтер dior,3327621,3327595.0,Хайлайтеры,3327548
4,10477,23532,2020-08-19,хайлайтер dior,3327621,3327595.0,Хайлайтеры,3327548


In [19]:
redirs.to_csv("./data/redirects_with_new_category_id.csv", index=False)

In [20]:
t = list(sorted(Counter(redirs['new_category_id']).items(), key=lambda x: -x[1]))
t[:30]

[(3327548, 491),
 (3328328, 71),
 (3505256, 17),
 (3416256, 8),
 (3327781, 8),
 (3328015, 6),
 (3427054, 5),
 (3328868, 5),
 (3327572, 5),
 (3328745, 5),
 (3417938, 4),
 (3329342, 4),
 (3329394, 4),
 (3328425, 4),
 (3328459, 3),
 (3329238, 3),
 (3328775, 2),
 (3328057, 2),
 (3328183, 2),
 (873818, 2),
 (3328799, 2),
 (3328517, 2),
 (3327913, 2),
 (3329009, 2),
 (3327959, 1),
 (3329045, 1),
 (3329119, 1),
 (3329292, 1),
 (3328132, 1),
 (3327840, 1)]

In [21]:
len(t)

777

In [22]:
a = sum([x[1] for x in t if x[1] > 4])
b = sum([x[1] for x in t])
a, b, a / b

(621, 1412, 0.4398016997167139)

In [23]:
len(redirs)

1412

**Промежуточный вывод:** если брать "макро-категории", для которых есть больше 4 примеров, то останется ~ 43% данных.
Вероятно имеет смысл оставшиеся малые классы определить в "other" или найти способ для аугментации малых классов

In [24]:
category_with_id(3327548)

,id,parent_id,site_id,external_id,name
363,3327548,3328184.0,420,3657,Dior


In [25]:
category_with_id(3328328)

,id,parent_id,site_id,external_id,name
1609,3328328,3328184.0,420,4297,Dior Backstage


In [26]:
category_with_id(3505256)

,id,parent_id,site_id,external_id,name
1069,3505256,3328184.0,420,5726,Moroccanoil


In [27]:
category_with_id(3329236)

,id,parent_id,site_id,external_id,name
473,3329236,3328184.0,420,5320,Dr. Vranjes Firenze
